In [1]:
from src.morse import Morse
from src.evaluation_and_conversion import label_txt_to_label_dict, compare_result_dict_to_groundtruth_label_dict

import os
import itertools
import csv

In [4]:
# pipeline old ridges Morse segmentation
folder = "../../Data/fumane/"
folder_data = folder + "curvature/"
folder_gt = folder + "groundtruths/txts/"
folder_results = folder + "results/results_old_ridges_morse/"

persistences = [0.04,0.06,0.08]
high_threshs = [0.05,0.06,0.07,0.08,0.09,0.1,0.11]
low_threshs = [0.04,0.05,0.06,0.07,0.08,0.09,0.1]
merge_threshs = [0.2,0.3,0.4,0.5,0.6,0.7]


files = []
for file_gt in os.listdir(folder_gt):
    file_id = file_gt.split("_")[0]
    file_loc = folder_data + str(file_id) + "/"
    for f in os.listdir(file_loc):
        if f.split(".")[-1] == "ply":
            files.append([file_loc + f, folder_gt + file_gt])

for data_file, gt_file in files:
    print("----------------------------------------------")
    print("Process ", gt_file.split("_")[0].split("/")[-1])

    gt_labels = label_txt_to_label_dict(gt_file)

    data = Morse()
    data.load_mesh_new(data_file, inverted=True)
    data.process_lower_stars()
    data.extract_morse_complex()
    data.reduce_morse_complex(data.range)

    best = 0
    best_seg = None
    best_param = None
    for pers in persistences:
        data.reduce_morse_complex(pers)
        data.extract_morse_cells(pers)
        for high, low, merge in list(itertools.product(high_threshs, low_threshs, merge_threshs)):
            if high > low and high-low < 0.04:
                seg_cell_labels = data.segmentation(pers, high, low, merge, minimum_labels=5, separatrix_type="reverse")
                seg_labels = {k: v.vertices for k,v in seg_cell_labels.Cells.items()}

                correctness = compare_result_dict_to_groundtruth_label_dict(seg_labels, gt_labels)

                if correctness > best:
                    best = correctness
                    best_seg = seg_labels
                    best_param = [pers, high, low, merge]


    data.plot_labels_txt(best_seg, 
                         folder_results+gt_file.split("_")[0].split("/")[-1]+"_"+str(best_param[0])+"P_"+str(best_param[1])+"H_"+str(best_param[2])+"L_"+str(best_param[3])+"M",
                         cell_structure=False)
    with open(folder_results+gt_file.split("_")[0].split("/")[-1]+"_best_result.txt", "w") as f:
        f.write("Best result: " + str(best) + "%\n")
        f.write("Params: "+str(best_param[0])+" P "+str(best_param[1])+" H "+str(best_param[2])+" L "+str(best_param[3])+" M")


----------------------------------------------
Process  31
label_txt_to_label_dict took 0.08730 seconds to execute!
load_mesh_new took 10.55280 seconds to execute!
process_lower_stars took 3.10522 seconds to execute!
extract_morse_complex took 4.54286 seconds to execute!
Persistence was high enough that this complex is maximally reduced.
reduce_morse_complex took 6.50885 seconds to execute!
reduce_morse_complex took 5.86871 seconds to execute!
extract_morse_cells took 8.99059 seconds to execute!
reduce_morse_complex took 7.42450 seconds to execute!
extract_morse_cells took 4.45245 seconds to execute!
reduce_morse_complex took 8.99523 seconds to execute!
extract_morse_cells took 5.24672 seconds to execute!
plot_labels_txt took 0.06815 seconds to execute!
----------------------------------------------
Process  9337
label_txt_to_label_dict took 0.10351 seconds to execute!
load_mesh_new took 15.78953 seconds to execute!
process_lower_stars took 4.11783 seconds to execute!
extract_morse_com

In [9]:
folder = "../../Data/fumane/"
folder_data = folder + "curvature/"
folder_gt = folder + "groundtruths/txts/"
folder_results = folder + "results/results_old_ridges_morse/"

result_table = []
for file in os.listdir(folder_results):
    if file.split("_")[-1] == "result.txt":
        with open(folder_results+file, "r") as f:
            for line in f:
                if line[0] == "B":
                    res = float(line.split()[-1][:-1])
                elif line[0] == "P":
                    pers = line.split()[1]
                    high = line.split()[3]
                    low = line.split()[5]
                    merge = line.split()[7]
            result_table.append([file.split("_")[0], float(res), float(pers), float(high), float(low), float(merge)])
import numpy as np
reav=0
pav=0
hav=0
lav=0
mav=0
for r in np.array(result_table):
    reav += float(r[1])
    pav += float(r[2])
    hav += float(r[3])
    lav += float(r[4])
    mav += float(r[5])
result_table.append(["Average:", reav/len(result_table), pav/len(result_table), hav/len(result_table), lav/len(result_table), mav/len(result_table)]) 

with open(folder_results + "total_results.csv", "w") as f_res:
    writer = csv.writer(f_res)
    writer.writerows(result_table)
            

In [ ]:
# pipeline new ridges Morse segmentation
folder = "../../Data/fumane/"
folder_data = folder + "curvature/"
folder_gt = folder + "groundtruths/txts/"
folder_results = folder + "results/results_new_ridges_morse/"

persistences = [0.04,0.06,0.08]
high_threshs = [0.05,0.06,0.07,0.08,0.09,0.1,0.11]
low_threshs = [0.04,0.05,0.06,0.07,0.08,0.09,0.1]
merge_threshs = [0.2,0.3,0.4,0.5,0.6,0.7]


files = []
for file_gt in os.listdir(folder_gt):
    file_id = file_gt.split("_")[0]
    file_loc = folder_data + str(file_id) + "/"
    for f in os.listdir(file_loc):
        if f.split(".")[-1] == "ply":
            files.append([file_loc + f, folder_gt + file_gt])

for data_file, gt_file in files:
    print("----------------------------------------------")
    print("Process ", gt_file.split("_")[0].split("/")[-1])

    gt_labels = label_txt_to_label_dict(gt_file)

    data = Morse()
    data.load_mesh_new(data_file, inverted=True)
    data.process_lower_stars()
    data.extract_morse_complex()
    data.reduce_morse_complex(data.range)

    best = 0
    best_seg = None
    best_param = None
    for pers in persistences:
        data.reduce_morse_complex(pers)
        data.extract_morse_cells(pers)
        for high, low, merge in list(itertools.product(high_threshs, low_threshs, merge_threshs)):
            if high > low and high-low < 0.04:
                seg_cell_labels = data.segmentation(pers, high, low, merge, minimum_labels=5, separatrix_type="reverse")
                seg_labels = {k: v.vertices for k,v in seg_cell_labels.Cells.items()}

                correctness = compare_result_dict_to_groundtruth_label_dict(seg_labels, gt_labels)

                if correctness > best:
                    best = correctness
                    best_seg = seg_labels
                    best_param = [pers, high, low, merge]


    data.plot_labels_txt(best_seg, 
                         folder_results+gt_file.split("_")[0].split("/")[-1]+"_"+str(best_param[0])+"P_"+str(best_param[1])+"H_"+str(best_param[2])+"L_"+str(best_param[3])+"M",
                         cell_structure=False)
    with open(folder_results+gt_file.split("_")[0].split("/")[-1]+"_best_result.txt", "w") as f:
        f.write("Best result: " + str(best) + "%\n")
        f.write("Params: "+str(best_param[0])+" P "+str(best_param[1])+" H "+str(best_param[2])+" L "+str(best_param[3])+" M")